In [1]:


class Plot: 
    ## default contructor
    def __init__(self, index, labels, line): 
        self.unique_id  = -1
        # Somewhere labels shall be constant in PlotsManager
        self.features   = []
        
        self.features = line.strip().split(',')
        assert len(self.features) == len(labels)
        assert labels[index] == "indexField"

        self.unique_id = int(self.features[index])     

    
    def addFeatures(self,labels, currentlabels, newfeatures):
        additionalLabels = []
        if len(self.features)<len(labels):
           num_empty_strings_needed = len(labels) - len(self.features)
           self.features.extend([""] * num_empty_strings_needed)
        for i in range(len(currentlabels)):
            if currentlabels[i] in labels:
                current_index = labels.index(currentlabels[i])
                self.features[current_index] = newfeatures[i]
            else : 
                additionalLabels.append(currentlabels[i])
                self.features.append(newfeatures[i])   
        

        return additionalLabels
    
    ## @param[in] id : int
    def isthis(self,id):
        return self.unique_id == id

    def print(self):
        print (self.unique_id, self.features)


    def getString(self):
        return  ",".join(str(self.features)) + "\n"


In [2]:
class PlotsManager:
   ## default contructor
   def __init__(self,csvfieldplotdata):
      self.labels     = [] 
      self.indexindex = -1
      self.plots      = []
      self.unwantedlabels = ["system:index", ".geo"]

      file = open(csvfieldplotdata, 'r')
      first_line = file.readline().strip()
      if not first_line:
         raise Exception("File is empty")
      # else file is not empty
      self.labels = first_line.strip().split(',')
      try:
         self.indexindex = self.labels.index("indexField")
      except ValueError:
         print('"indexField" not found in the list of labels')

      while True:
         line = file.readline()
         if not line:  
            break
         newplot = Plot(self.indexindex,self.labels,line)
         self.plots.append(newplot)

      file.close()   

      ## test print statements 
      #print (self.labels)
      #print (self.indexindex)
      #for plot in self.plots:
      #   plot.print()
      
       
       
   ## method that takes as input a unique id and text if plot already exist
   def getPlotIndex(self,id):
      idi =int(id)
      for i in range(len(self.plots)):
         if self.plots[i].isthis(idi):
            return i
      return -1 # "Plot not found, wrong index given"


   # @returns unwantedIndex the indices of the columns to be removed
   # @returns index_list the order the items will be added once the unwantedIndex are removed
   def find_indexes(self, B):
      index_list = []
      unwantedIndex = []

      for i, element in enumerate(B):
         if element in self.labels:
            index_list.append(self.labels.index(element))
         else:
            if element not in self.unwantedlabels: 
               self.labels.append(element)
               index_list.append(len(self.labels)-1)
            else :
               unwantedIndex.append(B.index(element))

      print(self.unwantedlabels)
      print("unwantedIndex ", unwantedIndex)
      print("index_list ", index_list)
      print(self.labels)
      print(B)
      return index_list, unwantedIndex
   
   
   ## method that add data of given CSV file to the relevant plots
   def addInfo(self,filename):
      file = open(filename, 'r')
      first_line = file.readline().strip()
      if not first_line:
         print ("WARNING: File ", filename," is empty")
         return
      # else file is not empty
      labels = first_line.strip().split(',')
      geo_index = None
      if ".geo" in labels:
         geo_index = labels.index(".geo")
      
     
      ilist, unwanted = self.find_indexes(labels)
      
      
      
      
      plottosatindex = self.labels.index("indexField")
      
      print (plottosatindex, labels)
      return
   
      while True:
         line = file.readline()
         if not line:  
            break
         linelist = line.strip().split(',')

         if geo_index is not None and geo_index < len(linelist):
            linelist = linelist[:geo_index]

         filtered_linelist = [linelist[i] for i in range(len(linelist)) if i not in unwanted_indexes]
        
         plotIndex = self.getPlotIndex(filtered_linelist[plottosatindex])         
         additionalLabels = self.plots[plotIndex].addFeatures(self.labels,filtered_labels,filtered_linelist)
         self.labels.append(additionalLabels)
      file.close()   
      return

   def print(self):
      print(self.labels)
      for plot in self.plots:
         plot.print()


   def printLabels(self):
      print(self.labels)


   def exportTo(self,csvfile):
      file = open(csvfile, "w")
      labels_string = ",".join(self.labels) + "\n"
      file.write(labels_string)
      for plot in self.plots:
         file.write(plot.getString())
      file.close()